## Übungsblatt 2

##### Einleitung + Packages
Okay Leute, ich habe keine Ahnung, ob ich die Aufgabenstellung richtig verstehe, aber *falls* ich das tue, dann brauchen wir eine Operation, die eine Zeile aus der CSV übergeben bekommt und dann überprüft, ob alle Noten über einem gewissen Pegel liegen.

Kopieren wir Sachen aus Blatt 1, die sich nicht ändern werden.

In [1]:
import os
import csv
import numpy as np
import re

BASE_DIR: str = os.getcwd()
CSV_DIR: str = os.path.join(BASE_DIR, 'csv')


def read_csv_data(file_name: str) -> list:
    csv_data: list = []

    if not os.path.exists(CSV_DIR):
        os.makedirs(CSV_DIR)

    if file_name.endswith('.csv') and os.path.exists(file_name):
        with open(file=file_name, mode='r') as csv_file:
            csv_in = csv.reader(csv_file, quotechar='"', delimiter=";")
            for row in csv_in:
                csv_data.append(row)
    else:
        raise Exception("Path given doesn't point to a .csv file")

    return csv_data


def write_csv_data(csv_data: list, file_name: str) -> None:
    if os.path.exists(file_name):
        os.remove(file_name)

    with open(file=file_name, mode='w', newline='') as csv_file:
        csv_writer = csv.writer(csv_file, quotechar='"', delimiter=";")
        for row in csv_data:
            csv_writer.writerow(row)

#### ANALYZE
Fangen wir zuerst mit der Implementierung des Analyze-Befehls. Wir brauchen eine zusätzliche Operation, die eine Liste als Parameter, sowie einen Pegel bekommt und daraus bestimmt, ob eine Zeile gültig ist und in die finale Datei übernommen werden soll oder nicht.

- Threshold ist ein Integer, den wir nutzen, um die Entscheidung treffen zu können wann eine Zeile ungültig ist, standardmäßig bekommt er den Wert 0.

- Row ist die Liste, die wir übergeben und die Zellen enthält.

- G_Columns ist ebenfalls eine Liste, die wir im letzten Aufgabenblatt implementiert haben, damit wir auf mögliche Änderungen in den Dateien reagieren können.

Die Logik ist ganz simpel: Wir iterieren durch die Zellen mit Noten und überprüfen, ob alle auch über dem Pegel liegen. Sollten sie diesen tun, dann geben wir am Ende, nachdem wir aus der For-Schleife rauskommen True zurück, sonst False.

In [2]:
def analyze_threshold(row_in: list, g_columns: list, threshold: int = 0) -> bool:
    for column_index in g_columns:
        if int(row_in[column_index]) < threshold:
            return False

    return True

Jetzt müssen wir noch die Operation an die nötige Stelle einfügen, zuerst machen wir aber die restlichen Operationen.

#### PROJECTION

Mit der Projection wollen wir uns bestimmte Zeilen ausgeben lassen, d.h. zum Beispiel nur die Noten der Schüler oder die Schulen auf die sie gehen.

- S_Columns sind die selektierten Spalten, die ausgegeben werden sollen
- Column_Names sind die Spaltennamen
- Row ist nach wie vor unsere Zeile.

Ebenfalls eine sehr simple Operation, wir haben unsere Zeile, wir überprüfen für jeden Spaltennamen, ob er mit den Select-Spalten übereinstimmt und fügen die Zeile mithilfe des Index aus dem Dictionary zu der Output Reihe hinzu.

In [3]:
def analyze_select(row_in: list, s_columns: list, column_names: dict) -> list:
    row_out: list = []
    for column_key in column_names:
        if column_key in s_columns:
            row_out.append(row_in[column_names[column_key]])

    if row_out:
        return row_out
    else:
        raise DTBException("Die Spaltensuche ergab keine Treffer, korrigieren Sie Ihre Angaben!")

#### Implementieren

Nun implementieren wir die Operationen in unsere Hauptoperation.

Zuerst machen wir eine eigene Klasse, die von Exception erbt. Sie soll es erleichtern Fehler aufzufangen.

Außer g_columns, die die Indexe speichern an denen Noten eingetragen stehen, speichern wir nun ebenfalls alle Spalten in einem Dictionary nach der Form {Spaltenname: Index}. Zunächst überprüfen wir, ob alle Noten über dem Pegel liegen, sonst müssen wir uns nicht Mal die Mühe machen, um den Median, etc. zu berechnen. Danach holen wir uns die Spalten raus, die wir in s_columns ausgewählt haben. Soll es keine Übereinstimmung geben, dann wird eine Exception gefangen.

In [4]:
class DTBException(Exception):
    pass

In [5]:
def analyze_student_results(csv_data: list, s_columns: list = None, threshold: int = 0) -> list:
    csv_processed: list = []
    g_columns: list = []
    column_names: dict = {}

    for idx, row in enumerate(csv_data):

        if idx == 0:
            for g_idx, cell in enumerate(row):
                column_names[cell] = g_idx

                if re.match(r"^G[0-9]$", cell):
                    g_columns.append(g_idx)

            try:
                csv_processed.append(analyze_select(row_in=row + ['G_MAX', 'G_MEAN'], column_names=column_names, s_columns=s_columns))
            except DTBException as derr:
                print(derr.with_traceback())

        else:
            if analyze_threshold(row_in=row, g_columns=g_columns, threshold=threshold):
                grades: list = []
                for g_column in g_columns:
                    grades.append(int(row[g_column]))

                g_max: int = np.max(grades)
                g_mean: float = round(np.average(grades), 2)

                select_row: list = row + [str(g_max), str(g_mean)]

                try:
                    csv_processed.append(analyze_select(row_in=select_row, column_names=column_names, s_columns=s_columns))
                except DTBException as derr:
                    print(derr.with_traceback())

    return csv_processed

#### Testen

Testen wir den Scheiß und sehen dass es funktioniert.

In [7]:
file_in = os.path.join(CSV_DIR, 'student-mat.csv')
student_amount = sum(1 for line in open(file_in)) - 1
s_columns = ['famsize', 'sex']

file_out = os.path.join(CSV_DIR, f'student-mat-out-total-{student_amount}.csv')

csv_data = read_csv_data(file_name=file_in)
csv_data_processed = analyze_student_results(csv_data=csv_data, s_columns=s_columns, threshold=4)
write_csv_data(csv_data=csv_data_processed, file_name=file_out)

Created by [Arnold Schwarzenegger](https://www.instagram.com/schwarzenegger/)